In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from pathlib import Path
import csv
import codecs


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

LR = 0.001
EPOCHES = 1
BATCH_SIZE = 256
 

In [2]:
class FashionMNISTDataset(Dataset):
    def __init__(self, csv_file, transform=None):
        data = pd.read_csv(csv_file)
        self.X = np.array(data.iloc[:, 1:]).reshape(-1, 1, 28, 28).astype(float)
        self.Y = np.array(data.iloc[:, 0])
        del data
        self.len = len(self.X)
     
    def __len__(self):
        return self.len
     
    def __getitem__(self, idx):
        item = self.X[idx]
        label = self.Y[idx]
        return (item, label)

    
train_dataset = FashionMNISTDataset(csv_file=  "fashion-mnist_train.csv")
test_dataset = FashionMNISTDataset(csv_file=  "fashion-mnist_isok.csv")

train_loader = DataLoader(dataset=train_dataset, batch_size=256, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=256, shuffle=False)

In [4]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=(5, 5), padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU()
        )
        self.pool1 = nn.MaxPool2d(2)
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=(3, 3)),
            nn.BatchNorm2d(32),
            nn.ReLU()
        )
        self.layer3 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=(3, 3)),
            nn.BatchNorm2d(64),
            nn.ReLU()
        )
        self.pool2 = nn.MaxPool2d(2)
        self.fc = nn.Linear(5 * 5 * 64, 10)
     
    def forward(self, x):
        out = self.pool1(self.layer1(x))
        out = self.pool2(self.layer3(self.layer2(out)))
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

cnn=CNN()
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(cnn.parameters(), lr=LR)

def data_write_csv("submission.csv", datas):#file_name为写入CSV文件的路径，datas为要写入数据列表
    file_csv = codecs.open(file_name,'w+','utf-8')#追加
    writer = csv.writer(file_csv, delimiter=' ', quotechar=' ', quoting=csv.QUOTE_MINIMAL)
    for data in datas:
        writer.writerow(data)
    print("保存文件成功，处理结束")

In [9]:
losses = []
for epoch in range(EPOCHES):
    for i, (images, labels) in enumerate(train_loader):
        images = images.float().to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = cnn(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        losses.append(loss.cpu().data.item())
        if (i + 1) % 100 == 0:
            print('Epoch : %d/%d, Iter : %d/%d, Loss : %.4f' % (
                epoch + 1, EPOCHES,
                i + 1, len(train_dataset) // BATCH_SIZE,
                loss.data.item()
            ))

Epoch : 1/1, Iter : 100/234, Loss : 0.3736
Epoch : 1/1, Iter : 200/234, Loss : 0.2860


In [14]:
f = open('submission.csv', 'w',newline='', encoding='utf-8') 
csv_write = csv.writer(f)
csv_write.writerow(['Id', 'Category'])

cnn.eval()
correct = 0
total = 0
for images,Id in test_loader:
    images = images.float().to(device)
    outputs = cnn(images).cpu()
    _, predicted = torch.max(outputs.data, 1)#将可信度最高的编号返回，最终将这个predicted
    abc=predicted.detach().numpy()
    abc1=Id.detach().numpy()
    print(abc)
    csv_write.writerow([abc1, predicted])
    total += labels.size(0)
    correct += (predicted == labels).sum()
 
print('CNN 测试集准确率：%.2f %%' % (100 * correct / total))
# 输出：CNN 测试集准确率：90.78 %
f.close()

tensor([2, 0, 4, 3, 7, 8, 9, 7, 6, 5, 3, 9, 4, 4, 5, 8, 8, 0, 6, 9, 9, 6, 0, 4,
        8, 3, 8, 9, 5, 1, 0, 9, 5, 8, 8, 6, 5, 0, 6, 4, 5, 0, 4, 1, 8, 2, 3, 4,
        2, 0, 4, 6, 8, 9, 6, 3, 9, 1, 1, 0, 6, 5, 8, 0, 6, 1, 5, 9, 5, 6, 4, 9,
        2, 8, 9, 7, 5, 5, 7, 8, 8, 4, 4, 9, 8, 2, 2, 4, 9, 0, 3, 0, 8, 1, 9, 2,
        9, 6, 6, 7])
[2 0 4 3 7 8 9 7 6 5 3 9 4 4 5 8 8 0 6 9 9 6 0 4 8 3 8 9 5 1 0 9 5 8 8 6 5
 0 6 4 5 0 4 1 8 2 3 4 2 0 4 6 8 9 6 3 9 1 1 0 6 5 8 0 6 1 5 9 5 6 4 9 2 8
 9 7 5 5 7 8 8 4 4 9 8 2 2 4 9 0 3 0 8 1 9 2 9 6 6 7]
CNN 测试集准确率：91.00 %
